In [8]:
import networkx as nx
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

DATA_PATH = "data/"

# Create network
- Load dataframe with shelves
- Implode dataframe to make shelves into edgelist
- Calculate assortativity
    - Genre
    - Degrees
- Get largest sub network
- Create communities
    - Color according to genre

In [25]:
# Just test can be removed
test_links_coloums = ["person", "book"]
test_links = [[0, 500], [0, 501], [0, 502],
              [1, 501], [1, 401], [1, 402]]
test_df = pd.DataFrame(test_links, columns=test_links_coloums)
imploded_df = test_df.groupby('person')['book'].apply(list).reset_index()
imploded_df

,person,book
0,0,"[500, 501, 502]"
1,1,"[501, 401, 402]"


In [28]:
H = nx.compose_all(map(nx.complete_graph, imploded_df.book))
print(H.nodes, H.edges)
print(H)

[500, 501, 502, 401, 402] [(500, 501), (500, 502), (501, 502), (501, 401), (501, 402), (401, 402)]
Graph with 5 nodes and 6 edges
